# Project Part F: Deployment

![](banner_project.jpg)

In [1]:
analyst = "Rachel Chen" # Replace this with your name

In [2]:
f = "setup.R"; for (i in 1:10) { if (file.exists(f)) break else f = paste0("../", f) }; source(f)
options(repr.matrix.max.rows=674)
options(repr.matrix.max.cols=200)
update_geom_defaults("point", list(size=1))                                

.................................................... start of document ....................................................

## Directions

### Objective

Recommend a portfolio of 12 company investments that maximizes 12-month return of an overall \$1,000,000 investment.

### Approach

Retrieve a transformed dataset about public company fundamentals and use it reproduce the construction of a selected model.

Retrieve an investment opportunities dataset, comprising fundamentals for some set of public companies over some one-year period.  Transform the representation of the investment opportunities to match the representation expected by the model, leveraging previous analysis.

Use the model to make predictions about the investment opportunities and accordingly recommend a portfolio of 12 company investments.

## Business Model


The business model is ...

$ \begin{align} profit = \left( \sum_{i \in portfolio} (1 + growth_i) \times allocation_i \right) - budget \end{align} $

<br>

$ profit\,rate = profit \div budget $


$ \begin{align} budget = \sum_{i \in portfolio} allocation_i \end{align} $

<br>

Business model parameters include ...

* Budget = \\$1,000,000: total investment to allocate across the companies in the portfolio
* Portfolio Size = 12: number of companies in the portfolio
* Allocations = \\$1,000,000 $\div$ 12 to each company: investments to allocate to specific companies in the portfolio 

Fill the portfolio with companies that have the highest predicted growths.

In [3]:
# Set the business parameters.

budget = 1000000
portfolio_size = 12
allocation = rep(budget/portfolio_size, portfolio_size)

layout(fmt(budget), fmt(portfolio_size), fmt(allocation))

budget
1000000
portfolio_size
12
allocation
83333.33
83333.33
83333.33
83333.33
83333.33
83333.33


## Data Retrieval

_<< Discuss this data retrieval. >>_

The whole process of data retrieval is done in the first and second part of the project, and the csv file "My Data.csv" was created in Part B of the project. In the following code cell, we load the data we created and present the first 6 rows of the dataset.

In [4]:
# Retrieve "My Data.csv".  This is the ORIGINAL model training data.
data = read.csv("My Data.csv", header=TRUE)

# Present a few rows ...
data[1:6,]

gvkey,tic,conm,PC1,PC2,prccq,growth,big_growth
1004,AIR,AAR CORP,3.4371231,-0.2260719,43.69,0.0507455507,NO
1045,AAL,AMERICAN AIRLINES GROUP INC,-12.0332067,0.8045109,32.11,-0.3828560446,NO
1050,CECE,CECO ENVIRONMENTAL CORP,3.9532234,-0.7553386,6.75,0.3157894737,YES
1062,ASA,ASA GOLD AND PRECIOUS METALS,3.6561434,-0.7981915,8.66,-0.2164739518,NO
1072,AVX,AVX CORP,2.9282228,-0.7104200,15.25,-0.1184971098,NO
1075,PNW,PINNACLE WEST CAPITAL CORP,0.3488491,1.1389605,85.20,0.0002347969,NO


## Build Model

_<< Discuss this model construction. >>_

We construct a linear regression model to predict growth given PC1 and PC2, based on the original model training data. The resulted model has an intercept of -0.1185887, a coefficient for PC1 with a value of 0.0002455 and a coefficient for PC2 with a value of 0.0006294.

In [5]:
# Construct a linear regression model to predict growth given PC1 and PC2, based on the
# ORIGINAL model training data.
# Present a brief summary of the model parameters.
model = lm(growth~PC1+PC2, data)
model


Call:
lm(formula = growth ~ PC1 + PC2, data = data)

Coefficients:
(Intercept)          PC1          PC2  
 -0.1185887    0.0002455    0.0006294  


## Investment Opportunities

_<< Discuss this handling of investment opportunties. >>_

We first read the dataset Investment Opportunities. As the size function shows, there are 918 observations and 680 variables in the dataset. Then, we partite the dataset by calendar quarter into four new datasets, one for each quarter. Additionally, we filter the observations to include only those with non-missing prccq. Also, we remove any observations about companies that reported more than once per quarter. In the same time, we change all the variable names by suffixing them with quarter information. For the next step, we consolidate all the data we have by company. We consolidate the four quarter datasets into one dataset, with one observation per company that includes variables for all four quarters and remove any observations with missing prccq.q4 values. The resulting dataset has 230 observations and 2711 variables. After that, we filter the data to include only those variables with at least 80% non-missing values in the original model training data (we did the selection in Part B of the project). There are 230 observations and 923 variables in the resulting dataset. Then, we impute the data using the same imputation values as computed for the original model training data. There are 230 observations and 923 variables in the resulting dataset. Next, we compute principal components using the centroids and weight matrix from the analysis of the original model training data and apply those principal components to only the numeric and integer variables used in the analysis of the original model training data. There are 230 observations and 737 variables in the resulting dataset. Finally, we combine the company information dataset and PC dataset and filter datasets as necessary to produce a new datset that includes all investment opportunities, but includes only predictor variables stored by previous analysis. The resulting dataframe has 230 observations and 5 variables. After all those procedures above, we have a clean dataset which can be used for model prediction in the later part of the project.  

### Retrieve Data

In [6]:
# Retrieve "Investment Opportunities.csv"
# Present the dataset size ...
datax = read.csv("Investment Opportunities.csv", header=TRUE)
size(datax)

observations,variables
918,680


### Partition Data by Calendar Quarter 

To partition the dataset by calendar quarter in which information is reported, first add a synthetic variable to indicate such.  Then partition into four new datasets, one for each quarter, and drop the quarter variables. Additionally, filter the observations to include only those with non-missing `prccq`.  Then remove any observations about companies that reported more than once per quarter.  Then change all the variable names (except for the `gvkey`, `tic`, and `conm` variables) by suffixing them with quarter information - e.g., in the Quarter 1 dataset, `prccq` becomes `prccq.q1`, etc.

In [7]:
# Partition the dataset as described.
datax$quarter = quarter(mdy(datax[,2]))

data.current.q1 = datax[(datax$quarter==1) & !is.na(datax$prccq), -ncol(datax)]
data.current.q2 = datax[(datax$quarter==2) & !is.na(datax$prccq), -ncol(datax)]
data.current.q3 = datax[(datax$quarter==3) & !is.na(datax$prccq), -ncol(datax)]
data.current.q4 = datax[(datax$quarter==4) & !is.na(datax$prccq), -ncol(datax)]

data.current.q1 = data.current.q1[!duplicated(data.current.q1$gvkey),]
data.current.q2 = data.current.q2[!duplicated(data.current.q2$gvkey),]
data.current.q3 = data.current.q3[!duplicated(data.current.q3$gvkey),]
data.current.q4 = data.current.q4[!duplicated(data.current.q4$gvkey),]

colnames(data.current.q1)[-c(1, 10, 12)] = paste0(colnames(data.current.q1)[-c(1, 10, 12)], ".q1")
colnames(data.current.q2)[-c(1, 10, 12)] = paste0(colnames(data.current.q2)[-c(1, 10, 12)], ".q2")
colnames(data.current.q3)[-c(1, 10, 12)] = paste0(colnames(data.current.q3)[-c(1, 10, 12)], ".q3")
colnames(data.current.q4)[-c(1, 10, 12)] = paste0(colnames(data.current.q4)[-c(1, 10, 12)], ".q4")

In [8]:
# Present the sizes of the data partitions
layout(fmt(size(data.current.q1)),
       fmt(size(data.current.q2)),
       fmt(size(data.current.q3)),
       fmt(size(data.current.q4)))

observations,variables
209,680
observations,variables
221,680
observations,variables
227,680
observations,variables
230,680


### Consolidate Data by Company

Consolidate the four quarter datasets into one dataset, with one observation per company that includes variables for all four quarters.  Remove any observations with missing `prccq.q4` values.

In [9]:
# Consolidate the partitions as described.
# How many observations and variables in the resulting dataset? 
m12 = merge(data.current.q1, data.current.q2, by=c("gvkey", "tic", "conm"), all=TRUE)
m34 = merge(data.current.q3, data.current.q4, by=c("gvkey", "tic", "conm"), all=TRUE)
data.current = merge(m12, m34, by=c("gvkey", "tic", "conm"), all=TRUE, sort=TRUE)
data.current = data.current[!is.na(data.current$prccq.q4),]

size(data.current)

observations,variables
230,2711


### Transform Representation of Data

In [10]:
# Filter the data to include only those variables with at least 80% non-missing values
# in the ORIGINAL model training data.
# How many observations and variables in the resulting dataset? 
#
# You can use the readRDS() function. 
prevars = readRDS("My Pre-Variables.rds")
dataori = data.current[, prevars]
size(dataori)

observations,variables
230,923


In [11]:
# Impute the data using the same imputation values as computed for the ORIGINAL model
# training data. 
# How many observations and variables in the resulting dataset? 
#
# You can use the readRDS() and put_impute() functions.
ml = readRDS("My Imputation.rds")
impute.data = put_impute(dataori, ml)
size(impute.data)

observations,variables
230,923


In [12]:
# Compute principal components using the centroids and weight matrix from the analysis
# of the ORIGINAL model training data.  Apply to only the (numeric and integer) variables
# used in the analysis of the ORIGINAL model training data. 
# How many observations and variables in the resulting dataset? 
#
# You can use the readRDS() and predict() functions.
# You can use rownames(pc$rotation) to get the (numeric and integer) variables. 
datanew = select_if(impute.data, is.numeric)

pc = readRDS("My PC.rds")
pctable = predict(pc, datanew)

size(pctable)

observations,variables
230,737


In [13]:
# Combine and filter datasets as necessary to produce a new datset that includes all investment
# opportunities, but includes only predictor variables stored by previous analysis. 
# How many observations and variables?
# Present the few few observations of the resulting dataset.
#
# You can use the readRDS() function.
postvar = readRDS("My Post-Variables.rds")
result = data.frame(gvkey = dataori[, "gvkey"], tic = dataori[, "tic"], conm = dataori[, "conm"], PC1 = pctable[, "PC1"], PC2 = pctable[, "PC2"])
size(result)
head(result, 6)

observations,variables
230,5


gvkey,tic,conm,PC1,PC2
1004,AIR,AAR CORP,3.472287,-0.08425766
1410,ABM,ABM INDUSTRIES INC,2.796849,-0.10000479
1562,AMSWA,AMERICAN SOFTWARE -CL A,3.986999,-0.65366925
1618,AXR,AMREP CORP,3.642196,-0.56307768
1632,ADI,ANALOG DEVICES,-4.313079,0.96351130
1686,APOG,APOGEE ENTERPRISES INC,3.491347,0.18518797


## Apply Model

_<< Discuss this application of the model. >>_

We use the model we made in part 4 of this assignment to predict growth of each investment opportunity. We add the predicted value into the Result dataset as a new column. Then, we sort the dataframe by the prediction column, putting higher predicted value rows at the front. After that, we choose the first 12 rows of the dataframe: the 12 investment opportunities for recommendation. The Portfolio dataframe consists the gvkey, tic, conm and budget allocation for each of the 12 investment opportunities. Finally, we store the portfolio recommendation and confirm that all the formats are correct. 

### Predict & Make Portfolio Recommendation

In [14]:
# Use the model to predict growths of each investment opportunity.
# Recommend a portfolio of allocations to 12 investment opportunities: gvkey, tic, conm, allocation
result$predict = predict(model, result[, c("PC1", "PC2")])
portfolio = result[order(result$predict,decreasing = TRUE), ][1:12, ][, c(1,2,3)]
portfolio$allocation = budget/12
fmt(portfolio, title = "portfolio")

gvkey,tic,conm,allocation
23809,AZO,AUTOZONE INC,83333.33
29692,WEBC,WEBCO INDUSTRIES INC,83333.33
3570,CBRL,CRACKER BARREL OLD CTRY STOR,83333.33
63172,FDS,FACTSET RESEARCH SYSTEMS INC,83333.33
64344,MTN,VAIL RESORTS INC,83333.33
178704,ULTA,ULTA BEAUTY INC,83333.33
65430,PLCE,CHILDRENS PLACE INC,83333.33
10549,THO,THOR INDUSTRIES INC,83333.33
3504,COO,COOPER COS INC (THE),83333.33
7921,NDSN,NORDSON CORP,83333.33


### Store Portfolio Recommendation

In [15]:
# Store portfolio recommendation

write.csv(portfolio, paste0(analyst, ".csv"), row.names=FALSE)

### Confirm That Format Is Correct

In [16]:
portfolio.retrieved = read.csv(paste0(analyst, ".csv"), header=TRUE)
opportunities = unique(read.csv("Investment Opportunities.csv", header=TRUE)$gvkey)

columns = all(colnames(portfolio.retrieved) == c("gvkey", "tic", "conm", "allocation"))
companies = all(portfolio.retrieved$gvkey %in% opportunities)
allocations = round(sum(portfolio.retrieved$allocation)) == budget
                         
check = data.frame(analyst, columns, companies, allocations)
fmt(check, "Portfolio Recommendation | Format Check")

analyst,columns,companies,allocations
Rachel Chen,TRUE,TRUE,TRUE


<font size=1;>
<p style="text-align: left;">
Copyright (c) Berkeley Data Analytics Group, LLC
<span style="float: right;">
Document revised June 21, 2020
</span>
</p>
</font>